# 📁 010_bronze/csv_export Documentation

This folder contains the initial ingestion pipelines responsible for loading raw CSV data dumps from the Landing Zone (`d_dnst_000_landing`) into the Bronze Layer (`d_dnst_010_bronze`). This done as a temporary measure while we are waiting for the integrations of these tables.

The primary function of these notebooks is to perform a direct read and overwrite operation (`spark.read.csv`) to persist the raw data in Delta Lake format, serving as the foundational source tables for subsequent cleaning and transformation.

## 🎯 **Source and Target**

* **Source Catalog/Schema:** `d_dnst_000_landing`.volumes (Manual Dump Paths)

* **Target Catalog/Schema:** `d_dnst_010_bronze`.csv\_export

* **Operation Type:** `overwrite`

## 📄 **Notebook Summary**

| Output Table (`target_table`) | Notebook Function | Source Data Type |
| :--- | :--- | :--- |
| `010_c4s_dump` | Loads raw C4S Accounts data. | C4S (Customer) |
| `010_c4s_business_partner_relationship_dump` | Loads raw C4S Business Partner Relationship data. | C4S (Relationship) |
| `010_knmt` | Loads raw SAP ECC Customer Master (KNMT) data. | SAP ECC |
| `010_marm` | Loads raw SAP ECC Material UoM (MARM) data. | SAP ECC |
| `010_uom_conversion` | Loads, cleans, and standardizes Unit of Measure conversion factors. | SAP BTP |

## ⚙️ **Special Processing: Unit of Measure (UOM) Conversion**

The `010_uom_conversion` notebook is the only file in this folder that performs complex transformations prior to writing to the Bronze layer.

1.  **Wholesaler Forward-Fill:** It uses a `Window` function with `last(..., ignorenulls=True)` to fill the null `Wholesaler` field with the previous non-null value, based on monotonic ID order. This corrects potential data formatting issues where the Wholesaler name is only specified once for a block of records.

2.  **String Cleaning:** Quotes (`'`) are removed from all string columns using `regexp_replace`.

3.  **Wholesaler Name Mapping:** The original `Wholesaler` names are standardized to clean, consistent names (e.g., `HANOSVETTEN` $\rightarrow$ `Hanos_5525672`) using a `when/otherwise` chain.

4.  **Manual Data Union:** Two manually defined rows for `Makady_5525924` UoM conversions are added to the DataFrame using `unionByName` to ensure critical conversion factors are present.

The final output table, `010_uom_conversion`, is the standardized mapping table used later in the Silver layer for sales record cleaning.